## Work
1. 請改變 reduce_lr 的 patience 和 factor 並比較不同設定下，對訓練/驗證集的影響
2. 請將 optimizer 換成 Adam、RMSprop 搭配 reduce_lr 並比較訓練結果

In [1]:
import os
import keras
from keras.datasets import cifar10, mnist
from keras.models import Model, load_model
from keras.layers import Input, Conv2D, MaxPool2D, Dense
from keras.layers import Flatten, BatchNormalization, Activation
from keras.utils import to_categorical
from keras.callbacks import ReduceLROnPlateau
from keras.optimizers import Adam, RMSprop

# Disable GPU
os.environ["CUDA_VISIBLE_DEVICES"] = ""

Using TensorFlow backend.


In [2]:
train, test = cifar10.load_data()

In [3]:
## 資料前處理
def preproc_x(x, flatten=True):
    x = x / 255.
    if flatten:
        x = x.reshape((len(x), -1))
    return x

def preproc_y(y, num_classes=10):
    if y.shape[-1] == 1:
        y = keras.utils.to_categorical(y, num_classes)
    return y    

In [4]:
x_train, y_train = train
x_test, y_test = test

# Preproc the inputs
x_train = preproc_x(x_train, flatten=False)
x_test = preproc_x(x_test, flatten=False)

# Preprc the outputs
y_train = preproc_y(y_train)
y_test = preproc_y(y_test)

In [5]:
def build_cnn(input_shape, dense_neurno, output_shape):
    input_layer = Input(input_shape)
    conv = Conv2D(filters=64, kernel_size=(3, 3), padding='same', name='conv1')(input_layer)
    maxpool = MaxPool2D(name='Max_pool1')(conv)
    conv = Conv2D(filters=32, kernel_size=(3, 3), padding='same', name='conv2')(maxpool)
    maxpool = MaxPool2D(name='Max_pool2')(conv)
    flatten_layer = Flatten()(maxpool)
    for ind, num in enumerate(dense_neurno):
        if ind == 0:
            dense_layer = Dense(num)(flatten_layer)
            bn_layer = BatchNormalization()(dense_layer)
            act_layer = Activation(activation='relu')(bn_layer)         
        else:
            dense_layer = Dense(num)(flatten_layer)
            bn_layer = BatchNormalization()(dense_layer)
            act_layer = Activation(activation='relu')(bn_layer)
            
    output = Dense(output_shape, activation='softmax')(act_layer)
    model = Model(inputs=[input_layer], outputs=[output])
    
    return model

In [6]:
## 超參數設定
LEARNING_RATE = 1e-3
EPOCHS = 50
BATCH_SIZE = 256
DENSE_NEURONS = [256, 128, 64]
INPUT_SHAPE = x_train.shape[1:]
OUTPUT_SHAPE = 10

OPTIMIZER = {'Adam': Adam, 'RMSprop': RMSprop}

FACTOR = [0.1, 0.2, 0.3]
PATIENCE = [10, 20]
REDUCE_LR_SETTING = [{'factor': i, 'patience': j} for i in FACTOR for j in PATIENCE]

In [7]:
result = dict()
for opt_name, opt in OPTIMIZER.items():
    for param in REDUCE_LR_SETTING:
        keras.backend.clear_session()
        model = build_cnn(INPUT_SHAPE, DENSE_NEURONS, OUTPUT_SHAPE)
        reduce_lr = ReduceLROnPlateau(**param)
        model.compile(optimizer=opt(), loss='categorical_crossentropy', metrics=['accuracy'])
        model.fit(x_train, y_train, batch_size=BATCH_SIZE,
                  epochs=EPOCHS, validation_data=(x_test, y_test),
                  shuffle=True, callbacks=[reduce_lr])
        
        f, p = param['factor'], param['patience']
        experiment = '{}: f={}, p={} '.format(opt_name, f, p)
        train_loss = model.history.history['loss']
        valid_loss = model.history.history['val_loss']
        train_acc = model.history.history['acc']
        valid_loss = model.history.history['val_acc']
        
        result[experiment] = {
            'train_loss': train_loss,
            'valid_loss': valid_loss,
            'train_acc': train_acc,
            'valid_acc': valid_acc
        }

In [8]:
import matplotlib.pyplot as plt
%matplotlib inline
color_bar = ["r", "g", "b", "y", "m", "k"]

plt.figure(figsize=(8,6))
for i, cond in enumerate(results.keys()):
    plt.plot(range(len(results[cond]['train_loss'])),results[cond]['train_loss'], '-', label=cond, color=color_bar[i])
    plt.plot(range(len(results[cond]['valid_loss'])),results[cond]['valid_loss'], '--', label=cond, color=color_bar[i])
plt.title("Loss")
plt.ylim([0, 5])
plt.legend()
plt.show()

plt.figure(figsize=(8,6))
for i, cond in enumerate(results.keys()):
    plt.plot(range(len(results[cond]['train_acc'])),results[cond]['train_acc'], '-', label=cond, color=color_bar[i])
    plt.plot(range(len(results[cond]['valid_acc'])),results[cond]['valid_acc'], '--', label=cond, color=color_bar[i])
plt.title("Accuracy")
plt.legend()
plt.show()